# Problem 1

In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as Data
import numpy as np
import h5py

### Obtaining Data

In [2]:
# This reads the matlab data from the .mat file provided
class MatRead(object):
    def __init__(self, file_path):
        super(MatRead).__init__()

        self.file_path = file_path
        self.data = h5py.File(self.file_path)

    def get_strain(self):
        strain = np.array(self.data['strain']).transpose(2,0,1)
        return torch.tensor(strain, dtype=torch.float32)

    def get_stress(self):
        stress = np.array(self.data['stress']).transpose(2,0,1)
        return torch.tensor(stress, dtype=torch.float32)

In [3]:
A = MatRead('Data/Material_A.mat')
stress_A = A.get_stress()
strain_A = A.get_strain()

B = MatRead('Data/Material_B.mat')
stress_B = B.get_stress()
strain_B = B.get_strain()

C = MatRead('Data/Material_C.mat')
stress_C = C.get_stress()
strain_C = C.get_strain()

In [4]:
# shape is (n_samples, n_step, n_dir)

# n_samples = number of samples
# n_dir = directiona of measurements
# n_step = step in the loading path

print(stress_A.shape)
print(stress_B.shape)
print(stress_C.shape)

print(strain_A.shape)
print(strain_B.shape)
print(strain_C.shape)

# Some entry of Material A is empty
# Which means no strain in the perp direction

# Material C is a case where the dataset only includes 1D uni-axial deformation data 

torch.Size([1100, 50, 6])
torch.Size([1100, 50, 6])
torch.Size([1100, 50, 1])
torch.Size([1100, 50, 6])
torch.Size([1100, 50, 6])
torch.Size([1100, 50, 1])


In [5]:
# Lets start with Material A
stress = stress_A
strain = strain_A

In [6]:
# Split data into train and test
# Use 75% data for training
# Use 25% data for testing

num_samples, num_steps, num_dir = stress.shape

ntrain = int(np.round(num_samples * 0.75))
ntest =  num_samples - ntrain

print(ntrain, ntest)

train_strain = strain[:ntrain, :, :]
train_stress = stress[:ntrain, :, :]
test_strain = strain[:ntest, :, :] 
test_stress = strain[:ntest, :, :]

print(train_strain.shape)
print(test_strain.shape)
print(train_stress.shape)
print(test_stress.shape)

825 275
torch.Size([825, 50, 6])
torch.Size([275, 50, 6])
torch.Size([825, 50, 6])
torch.Size([275, 50, 6])


In [ ]:
### Feature Scaling (Normalisation)

# IS THIS CORRECT?

class DataNormalizer(object):
    def __init__(self, data, epsilon=1e-8):
        self.data = data
        self.epsilon = epsilon

    def minMax(self):
        min_vals = self.data.min(dim=0, keepdim=True).values.min(dim=1, keepdim=True).values
        max_vals = self.data.max(dim=0, keepdim=True).values.max(dim=1, keepdim=True).values
        range_vals = max_vals - min_vals 
        return (self.data - min_vals) / range_vals + self.epsilon

    def standardisation():
        pass


In [41]:
# Normalize your data
strain_normalised_data = DataNormalizer(strain).minMax()
train_strain_encode =  strain_normalised_data[:ntrain, :, :]
test_strain_encode  = strain_normalised_data[:ntest, :, :]

stress_normalised_data  =  DataNormalizer(stress).minMax()
train_stress_encode = stress_normalised_data[:ntrain, :, :]
test_stress_encode  = stress_normalised_data[:ntest, :, :]

print(train_strain_encode.shape)
print(train_stress_encode.shape)
print(test_strain_encode.shape)
print(test_stress_encode.shape)

torch.Size([825, 50, 6])
torch.Size([825, 50, 6])
torch.Size([275, 50, 6])
torch.Size([275, 50, 6])
